In [109]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

from skimage.measure import label, regionprops
from skimage.segmentation import clear_border

import math

In [2]:
#Import simulation
path = '/project/s916/davidle/RCE-MIP/simulations/RCE_300_3km_206x206/' #Path for Daint

In [223]:
#Read Data
d2D = xr.open_mfdataset(path+'output/lfff[2,3,4]???0000.nc', combine='by_coords')

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
#Mask of all moist regions
bFMSE = np.where(d2D.FMSE > np.median(d2D.FMSE, axis=(1,2))[:, np.newaxis, np.newaxis], 1, 0)
bFMSE = np.tile(bFMSE[:,3:-3,3:-3],(2,2))

In [ ]:
for t in range(np.shape(bFMSE)[0]):
    bFMSE[t,:,:] = clear_border(bFMSE[t,:,:])

In [ ]:
plt.figure(figsize=(20, 20))
for t in range(0,500,5):
    plt.subplot(20,10,t/5+1)
    plt.contourf(bFMSE[t,:,:], cmap = 'viridis')
    plt.contour(bFMSE[t,:,:], colors = 'k')

    labels = label(bFMSE[t,:,:], neighbors=8)
    props = regionprops(labels)
    
    area = np.array([])
    for p in range(len(props)):
        area = np.append(area,props[p].area)
        
    if np.size(area) == 0:
        continue

    biggest = np.argmax(area)
                
    y0, x0 = props[biggest].centroid
    orientation = props[biggest].orientation
    x1 = x0 + math.cos(orientation) * 0.5 * props[biggest].major_axis_length
    y1 = y0 - math.sin(orientation) * 0.5 * props[biggest].major_axis_length
    x2 = x0 - math.sin(orientation) * 0.5 * props[biggest].minor_axis_length
    y2 = y0 - math.cos(orientation) * 0.5 * props[biggest].minor_axis_length

    plt.plot((x0, x1), (y0, y1), '-r', linewidth=2.5)
    plt.plot((x0, x2), (y0, y2), '-r', linewidth=2.5)
    plt.plot(x0, y0, '.g', markersize=15)
        

In [ ]:
alpha = np.array([])
for t in range(np.shape(bFMSE)[0]):
    labels = label(bFMSE[t,:,:], neighbors=8)
    props = regionprops(labels)
    
    area = np.array([])
    for p in range(len(props)):
        area = np.append(area,props[p].area)

    if np.size(area) != 0:
        alpha = np.append(alpha, props[np.argmax(area)].eccentricity)
    else:
        alpha = np.append(alpha, np.nan)

In [ ]:
fig = plt.figure(figsize=(20, 10))
#plt.plot(np.arange(30), alpha)

plt.subplot(1,2,1)
plt.plot(np.arange(len(alpha)), alpha)
plt.ylabel('alpha | 0 is circle')

plt.subplot(1,2,2)
plt.scatter(alpha, np.gradient(alpha))
plt.xlabel('alpha')
plt.ylabel('dalpha / dt')
#plt.xscale('log')
#plt.yscale('log')
